In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np

DATA PREPARATION

In [2]:
df = pd.read_csv('NFL PBP.csv')

C:\Users\m224956\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.shape

(449371, 255)

In [4]:
df.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


In [5]:
df['game_id'] = df['game_id'].astype(str)
df['desc'] = df['desc'].astype(str)

# clean the data
for i in range(len(df['game_id'])):
    df.at[i, 'Year'] = df['game_id'][i][0:4] # will give the year of the game
    
for i in range(len(df['desc'])):
    if df['desc'][i].count('(') >= 1:
        # access the first letter before period and the word after (this will give full name; all in the description 'desc')
        number = df['desc'][i].find('.') 
        df.at[i, 'Player'] = df['desc'][i][number-1] + ' ' + df['desc'][i][number+1:].split(' ')[0]
    else:
        pass # do nothing

nfl_data = df[['game_seconds_remaining', 
               'ydstogo',
               'posteam_score',
               'defteam_score',
               'down',
               'shotgun',
               'play_type',
               'qb_kneel',
               'qb_spike',
               'yards_gained',
               'Player',
               'Year']].query(
                '(play_type == "pass") or (play_type == "run")').dropna()
nfl_data.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,down,shotgun,play_type,qb_kneel,qb_spike,yards_gained,Player,Year
1,3593.0,10,0.0,0.0,1.0,0,pass,0,0,5.0,B Roethlisberger,2009
2,3556.0,5,0.0,0.0,2.0,0,run,0,0,-3.0,W Parker,2009
3,3515.0,8,0.0,0.0,3.0,1,pass,0,0,0.0,B Roethlisberger,2009
5,3496.0,10,0.0,0.0,1.0,0,run,0,0,0.0,C Johnson,2009
6,3460.0,10,0.0,0.0,2.0,0,pass,0,0,4.0,K Collins,2009


In [6]:
nfl_data.shape

(318347, 12)

In [7]:
# create dictionary with all of the passing dataframes
years = list(range(2009,2019,1))

d = {}
for year in years:
    df = pd.read_csv(f'Pass {year}.csv')
    d[year] = df[['Player', 'Age', 'QBR', 'Y/G', 'Rate']]

In [8]:
d[2009].head()

,Player,Age,QBR,Y/G,Rate
0,Matt Schaub*\SchaMa00,28,73.8,298.1,98.6
1,Peyton Manning*+\MannPe00,33,81.3,281.3,99.9
2,Tom Brady*\BradTo00,32,73.2,274.9,96.2
3,Jay Cutler\CutlJa00,26,50.3,229.1,76.8
4,Tony Romo*\RomoTo00,29,64.5,280.2,97.6


In [9]:
def pass_data_cleaning(dic, lis):
    for year in lis:
        for i in range(len(dic[year]['Player'])):
            # if there's a *, grab the name before that *
            if dic[year]['Player'][i].count('*') >= 1:
                number = dic[year]['Player'][i].find('*')
                dic[year].at[i, 'Player'] = dic[year]['Player'][i][:number]
            # if there's a \, grab the name before that
            else:
                number = dic[year]['Player'][i].find('\\')
                dic[year].at[i, 'Player'] = dic[year]['Player'][i][:number]

        # make it so that the name matches the main dataframe, with just one letter for the first name
        for k in range(len(dic[year]['Player'])):
            dic[year].at[k, 'Player'] = list(dic[year]['Player'][k].split(' ')[0])[0] + ' ' + dic[year]['Player'][k].split(' ')[1]
            
        dic[year]['Year'] = year
            
    return dic

In [10]:
# pass the dictionary we created two cells above into the cleaning function
cleaned_dict = pass_data_cleaning(d, years)

<ipython-input-9-fcf6eee7d81f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dic[year]['Year'] = year


In [11]:
cleaned_dict[2009].head()

,Player,Age,QBR,Y/G,Rate,Year
0,M Schaub,28,73.8,298.1,98.6,2009
1,P Manning,33,81.3,281.3,99.9,2009
2,T Brady,32,73.2,274.9,96.2,2009
3,J Cutler,26,50.3,229.1,76.8,2009
4,T Romo,29,64.5,280.2,97.6,2009


In [12]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver

In [13]:
def rush_data_forming_and_cleaning(lis):
    d_2 = {}
    for year in lis:
        driver = webdriver.Firefox('./')
        driver.get(f'https://www.footballdb.com/statistics/nfl/player-stats/rushing/{year}/regular-season')

        source = driver.page_source
        soup = BeautifulSoup(source)

        table = soup.find('table', {'class':'statistics scrollable'})

        # gather the tags that containt the name of the running backs
        data1 = table.find_all('span', {'class':'hidden-xs'})

        # create a list of running backs; append what is inside of the 'a' tag
        running_backs = []
        for data in data1:
            name = (data.find('a'))
            running_backs.append(name.text)

        # gather the tags that contain the avg yards and yards per game of the running backs
        # create respective lists for those datapoints
        data2 = table.find_all('td')
        average_yds = data2[5::10]
        for i in range(len(average_yds)):
            average_yds[i] = average_yds[i].text
        average_yds = [float(i) for i in average_yds]

        ypg = data2[6::10]
        for i in range(len(ypg)):
            ypg[i] = ypg[i].text
        ypg = [float(i) for i in ypg]

        # form the running back dataframe for these lists
        RB_df = pd.DataFrame(
            {'Player': running_backs,
             'AVG': average_yds,
             'YPG': ypg,
             'Year': year
            })

        for k in range(len(RB_df['Player'])):
            RB_df.at[k, 'Player'] = RB_df['Player'][k].split(' ')[0][0] + ' ' + RB_df['Player'][k].split(' ')[1]
            
        d_2[year] = RB_df
        
    return d_2

In [14]:
d_2 = rush_data_forming_and_cleaning(years)

In [15]:
d_2[2009].head()

,Player,AVG,YPG,Year
0,C Johnson,5.60,125.4,2009
1,S Jackson,4.37,94.4,2009
2,T Jones,4.24,87.6,2009
3,M Jones-Drew,4.46,86.9,2009
4,A Peterson,4.40,86.4,2009


In [16]:
# form the quarterback and running back dataframes into list of dataframes
qb_list = [d[i] for i in years]
rb_list = [d_2[i] for i in years]

In [17]:
# concatenate the lists into separate dataframes
qb_df = pd.concat(qb_list)
rb_df = pd.concat(rb_list)

In [18]:
qb_df['Year'] = qb_df['Year'].astype(str)
rb_df['Year'] = rb_df['Year'].astype(str)

In [19]:
nfl_data = nfl_data.merge(qb_df, on=['Player', 'Year'], how='left').merge(rb_df, on=['Player', 'Year'], how='left')

nfl_data.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,down,shotgun,play_type,qb_kneel,qb_spike,yards_gained,Player,Year,Age,QBR,Y/G,Rate,AVG,YPG
0,3593.0,10,0.0,0.0,1.0,0,pass,0,0,5.0,B Roethlisberger,2009,27.0,69.0,288.5,100.5,2.05,5.5
1,3556.0,5,0.0,0.0,2.0,0,run,0,0,-3.0,W Parker,2009,NaN,NaN,NaN,NaN,3.97,27.8
2,3515.0,8,0.0,0.0,3.0,1,pass,0,0,0.0,B Roethlisberger,2009,27.0,69.0,288.5,100.5,2.05,5.5
3,3496.0,10,0.0,0.0,1.0,0,run,0,0,0.0,C Johnson,2009,24.0,0.1,0.0,39.6,5.60,125.4
4,3496.0,10,0.0,0.0,1.0,0,run,0,0,0.0,C Johnson,2009,24.0,0.1,0.0,39.6,10.43,5.2


PREDICTING "YARDS_GAINED" GIVEN:
- game_seconds_remaining (from 3600 downwards)
- ydstogo (until a first down)
- down (the number, 1 to 4)
- shotgun (binary, yes or no)
- play_type (run or pass)
- qb_kneel or qb_spike (binary, yes or no)
- posteam_score and defteam_score (possessing and defensive)
- rushing average yards per game and per carry
- quarterback age, yards/game, QBR, and rating

In [20]:
nfl_pass_data = nfl_data[['game_seconds_remaining', 
               'ydstogo',
               'posteam_score',
               'defteam_score',
               'Age',
               'QBR',
               'Y/G',
               'Rate',
               'down',
               'shotgun',
               'play_type',
               'qb_kneel',
               'qb_spike',
               'yards_gained']].query('(play_type == "pass")').dropna()
nfl_pass_data.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,Age,QBR,Y/G,Rate,down,shotgun,play_type,qb_kneel,qb_spike,yards_gained
0,3593.0,10,0.0,0.0,27.0,69.0,288.5,100.5,1.0,0,pass,0,0,5.0
2,3515.0,8,0.0,0.0,27.0,69.0,288.5,100.5,3.0,1,pass,0,0,0.0
6,3460.0,10,0.0,0.0,37.0,41.1,175.0,65.5,2.0,0,pass,0,0,4.0
10,3384.0,10,0.0,0.0,27.0,69.0,288.5,100.5,1.0,0,pass,0,0,3.0
11,3348.0,7,0.0,0.0,27.0,69.0,288.5,100.5,2.0,0,pass,0,0,10.0


In [64]:
nfl_rush_data = nfl_data[['game_seconds_remaining', 
               'ydstogo',
               'posteam_score',
               'defteam_score',
               'AVG',
               'YPG',
               'down',
               'shotgun',
               'play_type',
               'qb_kneel',
               'qb_spike',
               'yards_gained']].query('(play_type == "run")').dropna()

nfl_rush_data.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,AVG,YPG,down,shotgun,play_type,qb_kneel,qb_spike,yards_gained
1,3556.0,5,0.0,0.0,3.97,27.8,2.0,0,run,0,0,-3.0
3,3496.0,10,0.0,0.0,5.60,125.4,1.0,0,run,0,0,0.0
4,3496.0,10,0.0,0.0,10.43,5.2,1.0,0,run,0,0,0.0
5,3496.0,10,0.0,0.0,10.67,2.0,1.0,0,run,0,0,0.0
7,3431.0,6,0.0,0.0,5.60,125.4,3.0,1,run,0,0,-2.0


In [24]:
# generate a seed; make results reproducable
np.random.seed(896)

NFL_pass = shuffle(nfl_pass_data)
NFL_pass.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,Age,QBR,Y/G,Rate,down,shotgun,play_type,qb_kneel,qb_spike,yards_gained
174565,2278.0,13,14.0,7.0,33.0,62.5,275.6,92.1,2.0,1,pass,0,0,4.0
56744,279.0,7,36.0,26.0,23.0,47.0,219.5,76.5,3.0,1,pass,0,0,0.0
277993,377.0,5,10.0,27.0,29.0,65.3,277.9,99.3,2.0,1,pass,0,0,12.0
317709,3004.0,10,0.0,0.0,21.0,26.6,162.7,66.7,1.0,0,pass,0,0,0.0
328511,1831.0,6,0.0,20.0,23.0,56.2,284.6,90.8,2.0,1,pass,0,0,7.0


In [66]:
# generate a seed; make results reproducable
np.random.seed(896)

NFL_rush = shuffle(nfl_rush_data)
NFL_rush.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,AVG,YPG,down,shotgun,play_type,qb_kneel,qb_spike,yards_gained
176362,352.0,7,41.0,10.0,3.92,20.8,2.0,0,run,0,0,-1.0
106976,2852.0,10,0.0,7.0,3.80,43.7,1.0,0,run,0,0,5.0
268634,2479.0,5,17.0,0.0,4.43,47.9,2.0,1,run,0,0,0.0
6567,505.0,10,20.0,19.0,4.44,61.6,1.0,0,run,0,0,0.0
183987,2802.0,10,10.0,10.0,3.92,20.8,1.0,1,run,0,0,2.0


PASS DATA

In [26]:
# feature, or input, dataframe
NFL_pass_x  = NFL_pass.drop('yards_gained', axis=1)
NFL_pass_x.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,Age,QBR,Y/G,Rate,down,shotgun,play_type,qb_kneel,qb_spike
174565,2278.0,13,14.0,7.0,33.0,62.5,275.6,92.1,2.0,1,pass,0,0
56744,279.0,7,36.0,26.0,23.0,47.0,219.5,76.5,3.0,1,pass,0,0
277993,377.0,5,10.0,27.0,29.0,65.3,277.9,99.3,2.0,1,pass,0,0
317709,3004.0,10,0.0,0.0,21.0,26.6,162.7,66.7,1.0,0,pass,0,0
328511,1831.0,6,0.0,20.0,23.0,56.2,284.6,90.8,2.0,1,pass,0,0


In [27]:
NFL_pass_x = NFL_pass_x.drop(['play_type'], axis=1)
NFL_pass_x.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,Age,QBR,Y/G,Rate,down,shotgun,qb_kneel,qb_spike
174565,2278.0,13,14.0,7.0,33.0,62.5,275.6,92.1,2.0,1,0,0
56744,279.0,7,36.0,26.0,23.0,47.0,219.5,76.5,3.0,1,0,0
277993,377.0,5,10.0,27.0,29.0,65.3,277.9,99.3,2.0,1,0,0
317709,3004.0,10,0.0,0.0,21.0,26.6,162.7,66.7,1.0,0,0,0
328511,1831.0,6,0.0,20.0,23.0,56.2,284.6,90.8,2.0,1,0,0


In [28]:
NFL_pass_x = NFL_pass_x.values
NFL_pass_x

array([[2.278e+03, 1.300e+01, 1.400e+01, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [2.790e+02, 7.000e+00, 3.600e+01, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [3.770e+02, 5.000e+00, 1.000e+01, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.310e+02, 1.000e+01, 1.900e+01, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [1.881e+03, 1.000e+01, 7.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [2.203e+03, 2.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [29]:
# label, or output, dataframe
NFL_pass_y = NFL_pass['yards_gained'].values
NFL_pass_y

array([  4.,   0.,  12., ...,   0.,   9., -11.])

In [30]:
#split the data
valid_split = 0.2
test_split = 0.1

In [31]:
# create the splits in the dataframe
v_split = int(NFL_pass.shape[0]*(1-valid_split-test_split))
t_split = int(NFL_pass.shape[0]*(1-test_split))

# assign training, validation, and test data
X_train, Y_train  = NFL_pass_x[:v_split], NFL_pass_y[:v_split]
X_valid, Y_valid  = NFL_pass_x[v_split:t_split], NFL_pass_y[v_split:t_split]
X_test , Y_test   = NFL_pass_x[t_split:], NFL_pass_y[t_split:]

In [32]:
#standardize the data
scaler = preprocessing.StandardScaler().fit(X_train[:,0:8]) # mean and variance calculated on specified columns of X_train
X_train[:,0:8] = scaler.transform(X_train[:,0:8])
X_valid[:,0:8] = scaler.transform(X_valid[:,0:8])
X_test[:,0:8]  = scaler.transform(X_test[:,0:8])  

In [33]:
# check that the proper data was standardized
X_train[0]

array([ 0.55456831,  1.02419449,  0.50622118, -0.51845532,  0.9294753 ,
        0.48688152,  0.77827633,  0.34886001,  2.        ,  1.        ,
        0.        ,  0.        ])

In [34]:
# import packages for the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [35]:
# initiate model
model = Sequential()

# add layers
model.add(Dense(50, 
                input_shape=(NFL_pass_x.shape[1],), 
                activation='tanh'))
model.add(Dense(50, 
                activation='tanh'))
model.add(Dense(50, 
                activation='tanh'))
model.add(Dense(1, 
                activation='linear'))

In [36]:
model.compile(loss='mean_squared_error', 
              optimizer=Adam(learning_rate=0.01))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                650       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 5,801
Trainable params: 5,801
Non-trainable params: 0
_________________________________________________________________


In [38]:
es = EarlyStopping(monitor='val_loss', 
                   patience=10, 
                   verbose=1,
                   restore_best_weights=True)

In [39]:
model.fit(X_train, Y_train, 
          epochs=1000, 
          callbacks=[es],
          batch_size=32, 
          shuffle=True,  
          validation_data=(X_valid, Y_valid))

Epoch 1/1000
4096/4096 [==============================] - 9s 2ms/step - loss: 102.7414 - val_loss: 104.2152
Epoch 2/1000
4096/4096 [==============================] - 7s 2ms/step - loss: 102.5705 - val_loss: 103.7323
Epoch 3/1000
4096/4096 [==============================] - 6s 1ms/step - loss: 102.4265 - val_loss: 104.0104
Epoch 4/1000
4096/4096 [==============================] - 10s 2ms/step - loss: 102.3488 - val_loss: 103.5628
Epoch 5/1000
4096/4096 [==============================] - 11s 3ms/step - loss: 102.3308 - val_loss: 103.5983
Epoch 6/1000
4096/4096 [==============================] - 13s 3ms/step - loss: 102.3046 - val_loss: 104.0590
Epoch 7/1000
4096/4096 [==============================] - 11s 3ms/step - loss: 102.2453 - val_loss: 103.6800
Epoch 8/1000
4096/4096 [==============================] - 10s 2ms/step - loss: 102.2231 - val_loss: 103.6670
Epoch 9/1000
4096/4096 [==============================] - 11s 3ms/step - loss: 102.2352 - val_loss: 103.8948
Epoch 10/1000
4096/409

In [41]:
model.evaluate(X_test, Y_test)

586/586 [==============================] - 1s 1ms/step - loss: 102.8387


102.83869171142578

RUSH DATA

In [75]:
# feature, or input, dataframe
NFL_rush_x  = NFL_rush.drop(['yards_gained', 'play_type'], axis=1)
NFL_rush_x.head()

,game_seconds_remaining,ydstogo,posteam_score,defteam_score,AVG,YPG,down,shotgun,qb_kneel,qb_spike
176362,352.0,7,41.0,10.0,3.92,20.8,2.0,0,0,0
106976,2852.0,10,0.0,7.0,3.80,43.7,1.0,0,0,0
268634,2479.0,5,17.0,0.0,4.43,47.9,2.0,1,0,0
6567,505.0,10,20.0,19.0,4.44,61.6,1.0,0,0,0
183987,2802.0,10,10.0,10.0,3.92,20.8,1.0,1,0,0


In [76]:
NFL_rush_x = NFL_rush_x.values
NFL_rush_x

array([[3.520e+02, 7.000e+00, 4.100e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.852e+03, 1.000e+01, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.479e+03, 5.000e+00, 1.700e+01, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.103e+03, 9.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.938e+03, 9.000e+00, 0.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [2.620e+02, 1.000e+01, 1.600e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [77]:
# label, or output, dataframe
NFL_rush_y = NFL_rush['yards_gained'].values
NFL_rush_y

array([-1.,  5.,  0., ...,  3.,  4., -1.])

In [79]:
#split the data
valid_split = 0.2
test_split = 0.1

In [80]:
# create the splits in the dataframe
v_split = int(NFL_rush.shape[0]*(1-valid_split-test_split))
t_split = int(NFL_rush.shape[0]*(1-test_split))

# assign training, validation, and test data
X_train, Y_train  = NFL_rush_x[:v_split], NFL_rush_y[:v_split]
X_valid, Y_valid  = NFL_rush_x[v_split:t_split], NFL_rush_y[v_split:t_split]
X_test , Y_test   = NFL_rush_x[t_split:], NFL_rush_y[t_split:]

In [81]:
#standardize the data
scaler = preprocessing.StandardScaler().fit(X_train[:,0:6]) # mean and variance calculated on specified columns of X_train
X_train[:,0:6] = scaler.transform(X_train[:,0:6])
X_valid[:,0:6] = scaler.transform(X_valid[:,0:6])
X_test[:,0:6]  = scaler.transform(X_test[:,0:6])  

In [83]:
# initiate model
model = Sequential()

# add layers
model.add(Dense(50, 
                input_shape=(NFL_rush_x.shape[1],), 
                activation='tanh'))
model.add(Dense(50, 
                activation='tanh'))
model.add(Dense(50, 
                activation='tanh'))
model.add(Dense(1, 
                activation='linear'))

In [84]:
model.compile(loss='mean_squared_error', 
              optimizer=Adam(learning_rate=0.01))

In [85]:
es = EarlyStopping(monitor='val_loss', 
                   patience=10, 
                   verbose=1,
                   restore_best_weights=True)

In [86]:
model.fit(X_train, Y_train, 
          epochs=1000, 
          callbacks=[es],
          batch_size=100, 
          shuffle=True,  
          validation_data=(X_valid, Y_valid))

Epoch 1/1000
940/940 [==============================] - 6s 5ms/step - loss: 41.8834 - val_loss: 39.4896
Epoch 2/1000
940/940 [==============================] - 3s 3ms/step - loss: 41.7157 - val_loss: 39.0930
Epoch 3/1000
940/940 [==============================] - 3s 3ms/step - loss: 41.7015 - val_loss: 39.1409
Epoch 4/1000
940/940 [==============================] - 3s 3ms/step - loss: 41.6839 - val_loss: 39.0489
Epoch 5/1000
940/940 [==============================] - 3s 4ms/step - loss: 41.6163 - val_loss: 39.0761
Epoch 6/1000
940/940 [==============================] - 4s 4ms/step - loss: 41.6239 - val_loss: 39.1106
Epoch 7/1000
940/940 [==============================] - 4s 5ms/step - loss: 41.5430 - val_loss: 39.0493
Epoch 8/1000
940/940 [==============================] - 5s 5ms/step - loss: 41.4930 - val_loss: 38.9639
Epoch 9/1000
940/940 [==============================] - 3s 3ms/step - loss: 41.4415 - val_loss: 39.0900
Epoch 10/1000
940/940 [==============================] - 4s 4ms/

In [87]:
model.evaluate(X_test, Y_test)

420/420 [==============================] - 1s 1ms/step - loss: 42.3815


42.38152313232422